https://www.analyticsvidhya.com/blog/2017/05/25-must-know-terms-concepts-for-beginners-in-deep-learning/ 


# Twython
## Loading Test Data Set

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns',100) # güncellenmiş max_columns

In [ ]:
#!pip install twython 
from twython import Twython
CONSUMER_KEY = "0pJRCT1emkOcrtxHl7lMN4N5A"
               
CONSUMER_SECRET = "IS4Gxcx1aQ1c5Hyei5PSePLS37IcVmYBT8Qt1t4IOxmLIxJD42"

twitter = Twython(CONSUMER_KEY,CONSUMER_SECRET)

In [ ]:
twitter

In [ ]:
dir(Twython)

In [ ]:
help(Twython).search

In [ ]:
results = twitter.search(q="data science", count = 200)
type(results)

In [ ]:
results

In [ ]:
all_items = results["statuses"]

In [ ]:
tweet_df = pd.DataFrame(all_items)
tweet_df.head()

In [ ]:
tweet_user = pd.DataFrame([d["user"] for d in results["statuses"]])
tweet_user.head()

In [ ]:
twitter_df = tweet_df.merge(tweet_user, left_index=True, right_index=True) # birleştirme

In [ ]:
twitter_df.shape

## Finding locations of most Tweets

In [ ]:
twitter_df.location.value_counts().plot(kind="bar", figsize=(10,6))

## Pulling 200 tweets with search term 'data science'

In [ ]:
user_screenname = []
user_location = []
user_name = []
user_tz = []
tweet_text = []
tweet_tm = []

In [ ]:
for status in twitter.search(q = "Datascience", result_type = "popular",
                             lang = "en",count = 200)["statuses"]:
    user = status["user"]["screen_name"]
    text = status["text"]
    tm = status["created_at"]        
    loc = status["user"]["location"]
    nm = status["user"]["name"]
    tz = status["user"]["time_zone"]
    
    print()
    print(user,"--",loc,"--",nm,"--",tm,"--",text)
    print()

    user_screenname.append(user)
    user_location.append(loc)
    user_name.append(nm)
    user_tz.append(tz)
    tweet_text.append(text)
    tweet_tm.append(tm)    

## Placing Tweets into a table

In [ ]:
test = twitter_df[["created_at_x","text"]]
test.head()

In [ ]:
test.drop("created_at_x",axis=1, inplace=True)
test.rename(columns={"text":"SentimentText"},inplace=True) # column name değiştirme
test.index.names=["ItemID"] # 
test.head()

## Exploring Data

In [ ]:
test.info()

In [ ]:
test.head()

In [ ]:
test.tail()

In [ ]:
from textblob import TextBlob
def detect_sentiment(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
test["Sentiment"]=test.SentimentText.apply(detect_sentiment)

In [ ]:
test.Sentiment.value_counts()

In [ ]:
df=test

In [ ]:
# Checking for duplicates
df.SentimentText.duplicated().sum()

In [ ]:
df.drop_duplicates(subset="SentimentText", inplace=True) # tekrar edenleri sil
df.info()

In [ ]:
# Removing special characters
df["removal_char"]=df["SentimentText"].str.replace("[^a-zA-Z0-9\s]",'') 
df.head()

In [ ]:
import nltk 
from nltk import word_tokenize

In [ ]:
# Lowercasing and tokenizing words
df.removal_char = df.removal_char.apply(lambda x: word_tokenize(x.lower()))

In [ ]:
TextBlob("I am very happy").sentiment.polarity

## Checking for distribution of negative and positive sentiments

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(x=df["Sentiment"])
plt.xticks(rotation=90);

In [ ]:
from textblob import TextBlob, Word
from nltk.corpus import stopwords 

In [ ]:
stop_words=list(stopwords.words("english"))
print(stop_words)

In [ ]:
# Removing Stopwords
df["stopword_removal"] = df["removal_char"].apply(lambda x: [word for word in x if word not in stop_words])
df.head()

In [ ]:
df["stopword_removal"] = df["stopword_removal"].apply(lambda x: " ".join(x)) # kelimleri birleştirdik

In [ ]:
from nltk import WordNetLemmatizer # kök bulma
df["stopword_removal"]=df["stopword_removal"].apply(lambda x : WordNetLemmatizer().lemmatize(x))

In [ ]:
#Calculating Polarity
df["sentiment_polarity"]=df["stopword_removal"].apply(lambda x : TextBlob(x).sentiment.polarity)

In [ ]:
#Calculating Subjectivity
df["sentiment_subjectivity"]=df["stopword_removal"].apply(lambda x : TextBlob(x).sentiment.subjectivity)

In [ ]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer

In [ ]:
vect=CountVectorizer()
X=vect.fit_transform(df["stopword_removal"])

In [ ]:
X.shape

In [ ]:
df.head(20)

## Training Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [ ]:
nb = MultinomialNB()
log = LogisticRegression()
dtc = DecisionTreeClassifier()

In [ ]:
def tweet_sentence(score): 
    if score>0.2: 
        return 2 
    elif score<-0.2: 
        return 0 
    else: 
        return 1 
df["class"]=df["sentiment_polarity"].apply(lambda x: tweet_sentence(x)) 

In [ ]:
x=df["stopword_removal"]
y=df["class"]

In [ ]:
x_dtm=vect.fit_transform(x)

In [ ]:
x_dtm.shape

In [ ]:
algorithms = [nb, log, dtc]
names = ['MultinomialNB', 'LogisticRegression', 'DecisionTreeClassifier'] 
def classification(): 
    accuracy = [] 
    precision = [] 
    recall = [] 
    f1 =[] 
    for i in range(len(algorithms)):
        algorithms[i].fit(x_dtm, y) 
    for i in range(len(algorithms)): 
        accuracy.append(accuracy_score(y, algorithms[i].predict(x_dtm))) 
        precision.append(precision_score(y, algorithms[i].predict(x_dtm),average="weighted")) 
        recall.append(recall_score(y, algorithms[i].predict(x_dtm),average="weighted")) 
        f1.append(f1_score(y, algorithms[i].predict(x_dtm),average="weighted")) 
   
    met = pd.DataFrame(columns = ['Accuracy', 'Precision', 'Recall', 'F1'], index = names) 
    met['Accuracy'] = accuracy 
    met['Precision'] = precision 
    met['Recall'] = recall 
    met["F1"] = f1 
    return met 

In [ ]:
classification() # train test split yapmadığımzdan, sonuçları ezberledi gibi yani

# RNN (Recurrent Neural Network)

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

%matplotlib inline

from matplotlib.pylab import rcParams
import numpy as np 
from datetime import datetime
import os

In [ ]:
training_set = pd.read_csv("Google_Stock_Price_Train.csv")

In [ ]:
training_set.head()

In [ ]:
training_set = training_set.iloc[:,1:2].values

In [ ]:
training_set # array oluypr artık

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler() # default is 0,1
training_set = sc.fit_transform(training_set)

In [ ]:
X_train = training_set[0:1257]
y_train = training_set[1:1258]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_train = np.reshape(X_train, (1257,1,1)) # data frame olmadığından böyle yaptık

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
regressor = Sequential()
regressor.add(LSTM(units=4, activation="sigmoid", input_shape=(None,1)))
regressor.add(Dense(units=1))
regressor.compile(optimizer="adam",loss="mean_squared_error")
regressor.fit(X_train, y_train, batch_size=32, epochs=200)

In [ ]:
test_set = pd.read_csv("Google_Stock_Price_Test.csv")
real_stock_price = test_set.iloc[:,1:2].values

In [ ]:
inputs = real_stock_price
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (20,1,1))
predicted_stock_price = regressor.predict(inputs)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.plot(real_stock_price, color="red", label="Real Google Stock Price")
plt.plot(predicted_stock_price, color="blue", label="Predicted Google Stock Price")
plt.title("Google Stock Price Prediction")
plt.xlabel("Time")
plt.ylabel("Google Time Price")
plt.legend()
plt.show()

In [ ]:
real_stock_price = pd.read_csv("Google_Stock_Price_Train.csv")
real_stock_price = real_stock_price.iloc[:,1:2].values

In [ ]:
predicted_stock_price_train = regressor.predict(X_train)
predicted_stock_price_train = sc.inverse_transform(predicted_stock_price_train)

In [ ]:
plt.plot(real_stock_price, color="red", label="Real Google Stock Price")
plt.plot(predicted_stock_price_train, color="blue", label="Predicted Google Stock Price")
plt.title("Google Stock Price Prediction")
plt.xlabel("Time")
plt.ylabel("Google Time Price")
plt.legend()
plt.show()

In [ ]:
regressor.save("my_model.h5")

In [ ]:
new_model = tf.keras.models.load_model("my_model.h5")

In [ ]:
predicted_stock_price = new_model.predict(inputs)

# Time Series Analysis

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

%matplotlib inline

from matplotlib.pylab import rcParams
import numpy as np 
from datetime import datetime
import os

In [ ]:
df=pd.read_csv("AirPassengers.csv")

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(df["#Passengers"])
plt.show

#### Checking how Stationary the data is

In [ ]:
from statsmodels.tsa.stattools import adfuller 

def test_stationarity(timeseries): 
  rolmean = timeseries.rolling(window=12).mean() 
  rolstd = timeseries. rolling(window=12).std()

  plt.figure(figsize=(8,5)) 
  orig = plt.plot(timeseries, color='blue',label='Original') 
  mean = plt.plot(rolmean, color='red', label='Rolling Mean') 
  std = plt.plot(rolstd, color='black', label = 'Rolling Std') 
  plt.legend(loc='best') 
  plt.title('Rolling Mean & Standard Deviation') 
  plt.show(block=False) 
  
  #Perform Dickey-Fuller test: 
  print ('Results of Dickey-Fuller Test:') 
  #iki çizgi arasında olup olmadığına bakmak için bu testi yapıyoruz
  dftest = adfuller(timeseries, autolag='AIC') 
  dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used']) 
  for key,value in dftest[4].items(): 
    dfoutput['Critical Value (%s)'%key] = value 
  print (dfoutput) 


In [ ]:
test_stationarity(df["#Passengers"]) # gün geçtikçe ortalama değişiyor (rolling mean)